In [1]:
import os
import requests
from urllib.parse import urljoin
import pandas as pd
import gzip
from urllib.parse import urljoin
import os

# Define the parameters
ensembl_version = 59  # Correct release version
organism_name = "arabidopsis_thaliana"
chromosome = "chromosome.1"  # Change this to download specific chromosomes
output_dir = "./output"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def download_gff(ensembl_version: int, organism_name: str, chromosome: str, output_dir: str) -> str:
    base_url = f"https://ftp.ensemblgenomes.ebi.ac.uk/pub/plants/release-{ensembl_version}/gff3/{organism_name}/"
    gff_file = f"{organism_name.capitalize()}.TAIR10.{ensembl_version}.{chromosome}.gff3.gz"
    full_url = urljoin(base_url, gff_file)
    local_path = os.path.join(output_dir, gff_file)

    # Download the file
    response = requests.get(full_url)
    if response.status_code == 200:
        with open(local_path, 'wb') as f:
            f.write(response.content)
        return local_path
    else:
        raise Exception(f"Failed to download GFF file for {organism_name} {chromosome} (status code: {response.status_code})")
# Test the download with chromosome 1
try:
    gff_file_path = download_gff(ensembl_version, organism_name, chromosome, output_dir)
    print(f"GFF file successfully downloaded to: {gff_file_path}")
except Exception as e:
    print(f"Error during download: {e}")


GFF file successfully downloaded to: ./output/Arabidopsis_thaliana.TAIR10.59.chromosome.1.gff3.gz


In [2]:
# Define the parse_gff function
def parse_gff(file_path: str) -> pd.DataFrame:
    with gzip.open(file_path, 'rt') as f:
        gff_data = pd.read_csv(f, sep='\t', comment='#', header=None, names=[
            'seqid', 'source', 'feature', 'start', 'end', 'score', 'strand', 'phase', 'attributes'
        ])
    
    genes = gff_data[gff_data['feature'] == 'gene']

    def parse_attributes(attributes):
        attributes_dict = dict(attr.split('=') for attr in attributes.split(';'))
        return attributes_dict.get('ID'), attributes_dict.get('Name')

    genes[['gene_id', 'gene_name']] = genes['attributes'].apply(lambda x: pd.Series(parse_attributes(x)))

    return genes[['seqid', 'start', 'end', 'gene_id', 'gene_name', 'strand']]

In [3]:
# Set parameters
ensembl_version = 59
organism_name = "arabidopsis_thaliana"
chromosome = "chromosome.1"
output_dir = "./output"

In [ ]:
gff_file_path = download_gff(ensembl_version, organism_name, chromosome, output_dir)
print(f"GFF file successfully downloaded to: {gff_file_path}")
genes_df = parse_gff(gff_file_path)

In [5]:
# Explore the DataFrame
# Display basic information about the DataFrame
print("\nDataFrame Info:")
print(genes_df.info())

print("\nFirst few rows of the DataFrame:")
display(genes_df.head())

print("\nDataFrame Description:")
display(genes_df.describe())

# Check for missing values
print("\nMissing Values:")
display(genes_df.isnull().sum())

# Gene lengths
genes_df['length'] = genes_df['end'] - genes_df['start']

# Top 10 longest genes
print("\nTop 10 Longest Genes:")
display(genes_df.nlargest(10, 'length'))

# Check for duplicate gene IDs
duplicate_ids = genes_df[genes_df.duplicated(subset='gene_id', keep=False)]
print(f"\nNumber of duplicate gene IDs: {len(duplicate_ids)}")
if len(duplicate_ids) > 0:
    print("Duplicate Gene IDs:")
    display(duplicate_ids)

# Check for genes without names
unnamed_genes = genes_df[genes_df['gene_name'].isnull()]
print(f"\nNumber of genes without names: {len(unnamed_genes)}")
if len(unnamed_genes) > 0:
    print("Sample of Unnamed Genes:")
    display(unnamed_genes.head())


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 7156 entries, 1 to 213201
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   seqid      7156 non-null   int64 
 1   start      7156 non-null   int64 
 2   end        7156 non-null   int64 
 3   gene_id    7156 non-null   object
 4   gene_name  7140 non-null   object
 5   strand     7156 non-null   object
dtypes: int64(3), object(3)
memory usage: 391.3+ KB
None

First few rows of the DataFrame:


,seqid,start,end,gene_id,gene_name,strand
1,1,3631,5899,gene:AT1G01010,NAC001,+
17,1,6788,9130,gene:AT1G01020,ARV1,-
135,1,11649,13714,gene:AT1G01030,NGA3,-
152,1,23121,31227,gene:AT1G01040,DCL1,+
245,1,31170,33171,gene:AT1G01050,PPa1,-



DataFrame Description:


,seqid,start,end
count,7156.0,7.156000e+03,7.156000e+03
mean,1.0,1.494342e+07,1.494586e+07
std,0.0,9.528129e+06,9.528120e+06
min,1.0,3.631000e+03,5.899000e+03
25%,1.0,6.066435e+06,6.068832e+06
50%,1.0,1.402051e+07,1.402270e+07
75%,1.0,2.373760e+07,2.373907e+07
max,1.0,3.042442e+07,3.042519e+07



Missing Values:


seqid         0
start         0
end           0
gene_id       0
gene_name    16
strand        0
dtype: int64


Top 10 Longest Genes:


,seqid,start,end,gene_id,gene_name,strand,length
114296,1,17732010,17758508,gene:AT1G48090,AT1G48090,-,26498
169043,1,25069428,25095638,gene:AT1G67120,AT1G67120,-,26210
146678,1,21746354,21766048,gene:AT1G58602,AT1G58602,+,19694
163088,1,24064822,24083428,gene:AT1G64790,ILA,-,18606
26374,1,2588854,2606892,gene:AT1G08260,TIL1,+,18038
121761,1,18522248,18539995,gene:AT1G50030,TOR,-,17747
169357,1,25100848,25117531,gene:AT1G67140,SWEETIE,-,16683
7844,1,712440,727553,gene:AT1G03060,SPI,-,15113
145817,1,21587060,21601740,gene:AT1G58250,SAB,-,14680
205734,1,29818970,29833016,gene:AT1G79280,NUA,-,14046



Number of duplicate gene IDs: 0

Number of genes without names: 16
Sample of Unnamed Genes:


,seqid,start,end,gene_id,gene_name,strand,length
23284,1,2295553,2296497,gene:AT1G07476,None,+,944
55995,1,5812728,5816662,gene:AT1G17000,None,+,3934
60323,1,6304360,6305689,gene:AT1G18320,None,+,1329
62926,1,6589835,6592762,gene:AT1G19090,None,+,2927
78450,1,8692630,8694705,gene:AT1G24530,None,+,2075
